# Data

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# for inline plots in jupyter
%matplotlib inline 
sns.set(color_codes=True) # settings for seaborn plotting style
sns.set(rc={'figure.figsize':(5,5)}) # settings for seaborn plot sizes

Read  the data to df from a csv file

In [11]:
df_train = pd.read_csv("./hotels_train.csv")

First we will get to know the data: the data type of each column and shape.

In [12]:
print("Shape: ", df_train.shape)
df_train.head()

Shape:  (27213, 18)


,ID,weekend_nights,week_nights,room_type,board_type,n_adults,n_less_12,n_more_12,booked_tour,n_requests,lead_time,purchase_type,n_p_cacellation,n_p_not_cacellation,repeated,price,date,is_canceled
0,INN09588,1,5,Room_Type 1,half board,2,0,0,0,2,34.0,Online,0,0,0,108.4,11/28/2018,0
1,INN07691,0,3,Room_Type 1,NaN,2,0,0,0,0,365.0,NaN,0,0,0,NaN,11/03/2018,1
2,INN32192,0,2,Room_Type 4,half board,1,0,0,0,1,148.0,Online,0,0,0,137.3,05/06/2018,0
3,INN32218,1,2,Room_Type 1,NaN,2,0,0,0,0,502.0,Offline,0,0,0,127.0,9/26/2018,1
4,INN02994,1,3,Room_Type 4,half board,2,0,1,0,2,32.0,Offline,0,0,0,110.0,10/19/2017,0


Atibruites

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27213 entries, 0 to 27212
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   27213 non-null  object 
 1   weekend_nights       27213 non-null  int64  
 2   week_nights          27213 non-null  int64  
 3   room_type            27213 non-null  object 
 4   board_type           19045 non-null  object 
 5   n_adults             27213 non-null  int64  
 6   n_less_12            27213 non-null  int64  
 7   n_more_12            27213 non-null  int64  
 8   booked_tour          27213 non-null  int64  
 9   n_requests           27213 non-null  int64  
 10  lead_time            26794 non-null  float64
 11  purchase_type        22366 non-null  object 
 12  n_p_cacellation      27213 non-null  int64  
 13  n_p_not_cacellation  27213 non-null  int64  
 14  repeated             27213 non-null  int64  
 15  price                23808 non-null 

In [14]:
df.describe(include='all')

,ID,weekend_nights,week_nights,room_type,board_type,n_adults,n_less_12,n_more_12,booked_tour,n_requests,lead_time,purchase_type,n_p_cacellation,n_p_not_cacellation,repeated,price,date,is_canceled
count,27213,27213.000000,27213.000000,27213,19045,27213.000000,27213.000000,27213.000000,27213.000000,27213.000000,26794.000000,22366,27213.000000,27213.000000,27213.000000,23808.000000,27213,27213.000000
unique,27213,NaN,NaN,7,4,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,553,NaN
top,INN09588,NaN,NaN,Room_Type 1,half board,NaN,NaN,NaN,NaN,NaN,NaN,Online,NaN,NaN,NaN,NaN,10/13/2018,NaN
freq,1,NaN,NaN,21084,14591,NaN,NaN,NaN,NaN,NaN,NaN,14306,NaN,NaN,NaN,NaN,188,NaN
mean,NaN,0.812810,2.197332,NaN,NaN,1.845221,0.052989,0.053357,0.031750,0.621100,102.952377,NaN,0.021975,0.155404,0.026421,123.455494,NaN,0.327674
std,NaN,0.869317,1.403576,NaN,NaN,0.519715,0.266150,0.268688,0.175336,0.785642,103.498942,NaN,0.346697,1.728693,0.160387,35.136566,NaN,0.469374
min,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,20.000000,NaN,0.000000
25%,NaN,0.000000,1.000000,NaN,NaN,2.000000,0.000000,0.000000,0.000000,0.000000,21.000000,NaN,0.000000,0.000000,0.000000,100.300000,NaN,0.000000
50%,NaN,1.000000,2.000000,NaN,NaN,2.000000,0.000000,0.000000,0.000000,0.000000,69.000000,NaN,0.000000,0.000000,0.000000,119.450000,NaN,0.000000
75%,NaN,2.000000,3.000000,NaN,NaN,2.000000,0.000000,0.000000,0.000000,1.000000,153.000000,NaN,0.000000,0.000000,0.000000,140.000000,NaN,1.000000


From the information above we can learn about missing values, attributes data type and unmatched data type.
- Missing values: for the attributes 'Age', 'Cabin' and 'Embarked'.
- Unmatched data type: for the attribute 'Fare' the data type is object and it should be a numeric ,we need to convert it.

Convert 'Fare' columns to numeric:

In [15]:
df['Fare'] = df['Fare'].str.extract('(\d+\.?\d*)', expand = True).astype(float)
df["Fare"].dtype

KeyError: 'Fare'

Split 'Cabin' column to sign and value:

In [ ]:
df[['Cabin_sign', 'Cabin_value']] = df['Cabin'].str.extract('([A-Za-z]+)(\d+\.?\d*)', expand = True)
df['Cabin_value'] = df['Cabin_value'].astype(float)

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

### Split to attributes and taget

In [ ]:
target =pd.DataFrame(df['Survived']) 
data = df.drop(['PassengerId', 'Survived','Cabin'], axis=1)
data

### Split to nominal and numerical attributes

In [ ]:
numeric_col = data.describe().columns # to get the numeric column
numeric_col

In [ ]:
numeric_data = data[numeric_col]
nominal_data = data.drop(numeric_col, axis=1)

## Numerical data
For numerical data we would like to know the range of values, whether there are outliers, the distribution and other statistics, so first we will draw the histograms.

In [ ]:
for col in numeric_col:
    sns.displot(numeric_data[col], bins=20, color='skyblue')

for col in numeric_col:
    sns.displot(numeric_data[col], bins=20, color='skyblue')

From the histograms above it looks like 'Age' is normal distributed whereas the other attributes are not. Moreover, we can see that 'Pclass', 'SibSp' and 'Parch' has only few values with importance to the order i.e. it's a categorical ordinal attribute.

#### Skweness

In [ ]:
for col in ['Age', 'Fare', 'Cabin_value']:
    print(col, ' skewness:', numeric_data[col].skew(axis = 0, skipna = True) )

As we can see at the graphs and from the skweness calculation, the 'fare' column have long right tail that indicate on asymmrtric distribution while 'Age' and 'Cabin_value' are more symmetric.

We can also look at the probability


In [ ]:
for col in ['Age', 'Fare', 'Cabin_value']:
    sns.displot(numeric_data[col], color='skyblue' ,stat="probability", discrete=True )

Check the values in 'Fare'

In [ ]:
print('min', numeric_data['Fare'].min())
print('max', numeric_data['Fare'].max())
print('values over 300', numeric_data[numeric_data['Fare']>300].shape)
print('values under 10', numeric_data[numeric_data['Fare']<10].shape)

Maybe there are outliers?

In [ ]:
data[data['Fare']>300]

### Categorial attributes

In [ ]:
cat_col = ['Pclass', 'SibSp','Parch']
for col in cat_col:
    print(numeric_data[col].value_counts().sort_index())
    print('-----------------------------')

#### Statistic information:
Describe function show statistic information for numerical data.

In [ ]:
numeric_data.describe() 

In [ ]:
sns.boxplot(y="Age", data=df)

In [ ]:
sns.boxplot(y="Fare", data=df)

In [ ]:
fare_up_to_100 = numeric_data[numeric_data['Fare']<=100]
fare_up_to_100.head()

In [ ]:
fare_up_to_100 = fare_up_to_100['Fare']

In [ ]:
sns.boxplot(y=fare_up_to_100)

In [ ]:
sns.displot(fare_up_to_100, color='skyblue', stat="probability", discrete=True )

## Nominal data
For nominal data we would like to know how many unique values there are and the distribution.

In [ ]:
nominal_data.info()

Describe function for nomiunal data show information about unique value, top value and the top value frequency.

In [ ]:
describe_df = nominal_data.describe()
describe_df

Adding the top value frequency in percent.

In [ ]:
describe_df.loc['%freq'] = describe_df.apply(lambda x: np.round(x.loc['freq']/891, decimals=3))
describe_df

#### Histograms
The columns 'Name' and 'Ticket' have to many unique values so there is no reason to look on their histograms.
Handale missing values - fill in 'NA' just to be able to see the histograms.

In [ ]:
nominal_data = nominal_data.fillna('NA')

In [ ]:
nominal_data['Embarked'].value_counts()

In [ ]:
dist_col = ['Sex', 'Embarked', 'Cabin_sign']
for col in dist_col:
    sns.displot(nominal_data, x=col, shrink=.8, color='skyblue')

## Target

In [ ]:
sns.displot(target, x='Survived', color='green', discrete=True)

#### Pie chart

In [ ]:
fig1, ax1 = plt.subplots()
ax1.pie(target['Survived'].value_counts(), labels=target['Survived'].unique(), autopct='%1.1f%%', shadow=True, startangle=90)
ax1.axis('equal') 
plt.show()

### Exploring the behavior of the attributes according to the target values

In [ ]:
df.groupby('Survived')['Age'].plot(kind='hist', bins=20, alpha=0.5, legend=True)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution by Survival')
plt.show()

#### Scatter plot

In [ ]:
colordict = {0: 'red', 1: 'blue'}

plt.figure(figsize=(15, 6))
plt.scatter(df.index, df['Fare'], c = df['Survived'].map(colordict))
plt.xlabel("Index")
plt.ylabel('Fare')

#### Boxplot graph 

In [ ]:
sns.set(rc={'figure.figsize':(5,8)})
sns.boxplot(y="Age", x="Sex", hue= "Survived", data=df, palette="Set3")